<a href="https://colab.research.google.com/github/abirinpajamas/adhd-prediction-ml-model/blob/main/adhd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv("adhd_data.csv")
df.head()

,Age,Gender,Educational_Level,Family_History,Sleep_Hours,Daily_Activity_Hours,Q1_1,Q1_2,Q1_3,Q1_4,...,Q2_8,Q2_9,Diagnosis_Class,Daily_Phone_Usage_Hours,Daily_Walking_Running_Hours,Difficulty_Organizing_Tasks,Focus_Score_Video,Daily_Coffee_Tea_Consumption,Learning_Difficulties,Anxiety_Depression_Levels
0,8,1,Primary,No,8,7,0,0,0,1,...,0,1,0,2,0.5,0,5,1,0,0
1,9,2,Primary,No,11,7,3,2,2,3,...,2,3,3,2,0.9,1,6,0,1,3
2,9,1,Primary,No,9,5,3,2,3,3,...,2,3,3,2,1.4,1,3,0,1,3
3,5,2,Kindergarten,Yes,7,11,3,3,3,2,...,0,0,2,6,0.6,1,6,0,1,1
4,13,1,Middle,No,3,0,3,2,3,3,...,1,0,2,4,1.0,1,5,1,1,2


In [9]:
df['Gender'] = df['Gender'].map({1: 'Male', 2: 'Female'}).astype('category')
df['Family_History']=df['Family_History'].astype('category')


In [10]:
df.head()

,Age,Gender,Educational_Level,Family_History,Sleep_Hours,Daily_Activity_Hours,Q1_1,Q1_2,Q1_3,Q1_4,...,Q2_8,Q2_9,Diagnosis_Class,Daily_Phone_Usage_Hours,Daily_Walking_Running_Hours,Difficulty_Organizing_Tasks,Focus_Score_Video,Daily_Coffee_Tea_Consumption,Learning_Difficulties,Anxiety_Depression_Levels
0,8,NaN,Primary,No,8,7,0,0,0,1,...,0,1,0,2,0.5,0,5,1,0,0
1,9,NaN,Primary,No,11,7,3,2,2,3,...,2,3,3,2,0.9,1,6,0,1,3
2,9,NaN,Primary,No,9,5,3,2,3,3,...,2,3,3,2,1.4,1,3,0,1,3
3,5,NaN,Kindergarten,Yes,7,11,3,3,3,2,...,0,0,2,6,0.6,1,6,0,1,1
4,13,NaN,Middle,No,3,0,3,2,3,3,...,1,0,2,4,1.0,1,5,1,1,2
